In [7]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [20]:
# 
train_dir = 'DS'
val_dir = 'val'
width, height = 224, 224


In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(width, height, 3)) #load VGG16 model without the top layer


In [5]:
# Freeze the base model
for i in base_model.layers:
    i.trainable = False

In [8]:
# create a new model
model= Sequential()
model.add(base_model) #add the VGG16 base model
model.add(Flatten()) #flatten the output of the VGG16 model
model.add(Dense(128, activation='relu')) #Dense layer 1
model.add(Dropout(0.5)) #dropout layer
model.add(Dense(1, activation='sigmoid')) #output layer 


In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,209 (68.38 MB)

 Trainable params: 3,211,521 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [16]:
#data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
      shear_range=0.2,
        zoom_range=0.2,
          horizontal_flip=True,
              rotation_range=90,
                width_shift_range=0.1,
                  height_shift_range=0.1,
                    fill_mode='nearest'
          )
val_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
#load the images from the directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(width, height),
    batch_size=32,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(width, height),
    batch_size=32,
    class_mode='binary')


Found 160 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [22]:
# train the model
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_generator,
    validation_steps=50
)

Epoch 1/10


/opt/conda/envs/quant/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-09-10 07:33:33.143379: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-10 07:33:33.610245: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-10 07:33:35.318094: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-09-10 07:33:35.396657: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-09-10 07:33:36.26182

  5/100 ━━━━━━━━━━━━━━━━━━━━ 18:08 11s/step - accuracy: 0.4417 - loss: 0.8353

2024-09-10 07:34:31.220710: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/conda/envs/quant/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 553ms/step - accuracy: 0.4674 - loss: 0.8114 - val_accuracy: 0.6667 - val_loss: 0.5867
Epoch 2/10


...

100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 546ms/step - accuracy: 0.6987 - loss: 0.5982 - val_accuracy: 0.8333 - val_loss: 0.4858
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 550ms/step - accuracy: 0.6380 - loss: 0.5963 - val_accuracy: 0.8750 - val_loss: 0.4577
Epoch 5/10


2024-09-10 07:38:13.791149: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 547ms/step - accuracy: 0.7116 - loss: 0.6014 - val_accuracy: 0.8333 - val_loss: 0.4525
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 550ms/step - accuracy: 0.7391 - loss: 0.5179 - val_accuracy: 0.8750 - val_loss: 0.4308
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 552ms/step - accuracy: 0.7003 - loss: 0.5451 - val_accuracy: 0.9167 - val_loss: 0.3707
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 551ms/step - accuracy: 0.7810 - loss: 0.4673 - val_accuracy: 0.8750 - val_loss: 0.3950
Epoch 9/10


2024-09-10 07:42:42.522896: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 552ms/step - accuracy: 0.7862 - loss: 0.5031 - val_accuracy: 0.9167 - val_loss: 0.3339
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 545ms/step - accuracy: 0.7645 - loss: 0.5019 - val_accuracy: 0.8750 - val_loss: 0.2972


In [23]:
# evaluate the model
loss , acc = model.evaluate(val_generator)
print(f'Loss: {loss}, Accuracy: {acc}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.8750 - loss: 0.2972
Loss: 0.29721131920814514, Accuracy: 0.875
